In [1]:
from shared_utils import catalog_utils
import pandas as pd
import geopandas as gpd

# Quickly get unique stops in D7 on the SHS

In [2]:
shared_data = catalog_utils.get_catalog(catalog_name='shared_data_catalog')

In [4]:
stops = shared_data.ca_transit_stops.read()

In [8]:
d7_shs = stops.query('district_name.str.contains("07 - Los Angeles") & meters_to_ca_state_highway <= 50')

In [45]:
d7_shs.head(3)

,org_id,agency,stop_id,stop_name,n_routes,route_ids_served,routetypes,n_arrivals,n_hours_in_service,meters_to_ca_state_highway,base64_url,district_name,geometry
114,recJk2sQrZAYUPMID,City of Glendale,190,Foothill at Hillard,3,"3, 33, 34",3,0,0,29.8,aHR0cHM6Ly9nbGVuZGFsZWNhLmdvdi9Ib21lL1Nob3dEb2...,07 - Los Angeles / Ventura,POINT (-118.21509 34.21114)
145,recJk2sQrZAYUPMID,City of Glendale,356,San Fernando at Elk,2,"12, 6",3,0,0,43.7,aHR0cHM6Ly9nbGVuZGFsZWNhLmdvdi9Ib21lL1Nob3dEb2...,07 - Los Angeles / Ventura,POINT (-118.26894 34.14146)
172,recJk2sQrZAYUPMID,City of Glendale,193,Foothill at La Canada,3,"3, 33, 34",3,0,0,10.0,aHR0cHM6Ly9nbGVuZGFsZWNhLmdvdi9Ib21lL1Nob3dEb2...,07 - Los Angeles / Ventura,POINT (-118.20499 34.20678)


In [43]:
agency_groups = d7_shs[['base64_url', 'agency']].drop_duplicates().groupby('base64_url').agency.unique()
agency_groups = pd.DataFrame(agency_groups).reset_index().rename(columns={'agency':'agency_group'})

In [34]:
df = d7_shs[['base64_url', 'agency']].drop_duplicates().groupby('base64_url').count()

In [35]:
df = df.reset_index().rename(columns={'agency':'count'})

In [36]:
d7_shs[['base64_url', 'agency']].drop_duplicates().merge(df, on='base64_url').sort_values('count', ascending=False)

,base64_url,agency,count
13,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,City of Camarillo,7
15,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,City of Moorpark,7
17,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,City of Simi Valley,7
11,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,Ventura County Transportation Commission,7
12,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,City of Thousand Oaks,7
16,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,City of Ojai,7
14,aHR0cHM6Ly9nb3ZjYnVzLmNvbS9ndGZz,Gold Coast Transit District,7
2,aHR0cDovL2d0ZnMuZ2lzLmZsaXgudGVjaC9ndGZzX2dlbm...,FlixBus,2
30,aHR0cHM6Ly9mb290aGlsbHRyYW5zaXQucmlkZXJhbGVydH...,City of Duarte,2
1,aHR0cDovL2d0ZnMuZ2lzLmZsaXgudGVjaC9ndGZzX2dlbm...,Greyhound,2


In [46]:
gdf = d7_shs.drop_duplicates(subset=['base64_url', 'stop_id']).merge(agency_groups, on='base64_url')

In [48]:
gdf.agency_group = gdf.agency_group.astype(str)

In [50]:
gdf.to_csv('d7_shs_stops_062025.csv')

In [51]:
from shared_utils import webmap_utils
from calitp_data_analysis import geography_utils

In [70]:
to_map = gdf.to_crs(geography_utils.CA_NAD83Albers_m)
to_map.geometry = to_map.buffer(50)

In [53]:
import branca

In [65]:
cmap = branca.colormap.step.RdPu_05
cmap.scale(vmin = 0, vmax = 50)

In [66]:
cmap.rgb_bytes_tuple(50)

(122, 1, 119)

In [59]:
webmap_utils.set_state_export?

Signature:
webmap_utils.set_state_export(
    gdf,
    bucket: str = 'calitp-map-tiles/',
    subfolder: str = 'testing/',
    filename: str = 'test2',
    map_type=None,
    map_title: str = 'Map',
    cmap: branca.colormap.ColorMap = None,
    color_col: str = None,
    legend_url: str = None,
    existing_state: dict = {},
    cache_seconds: int = 3600,
    manual_centroid: list = None,
) -> dict
Docstring:
Main function to use single page application webmap.

Applies light formatting to gdf for successful spa display. Will pass map_type
if supported by the spa and provided, otherwise leave as None for generic style.
GCS bucket is preset to a publically available one.
Supply cmap and color_col for coloring based on a Branca ColorMap and a column
to apply the color to.
Cache is 1 hour by default, can set shorter time in seconds for
"near realtime" applications (suggest 120) or development (suggest 0)

Returns dict with state dictionary and map URL. Can call multiple times and supply


In [71]:
to_map['color'] = [(122, 1, 119)] * to_map.shape[0]

In [72]:
webmap_utils.set_state_export(to_map, subfolder='adhoc', filename='d7_stops_062025_v2',
                             map_title="D7 Transit Stops within 50 meters of SHS, June 2025")

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


{'state_dict': {'name': 'null',
  'layers': [{'name': 'D7 Transit Stops within 50 meters of SHS, June 2025',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/adhocd7_stops_062025_v2.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}}],
  'lat_lon': (34.009967460117466, -118.34760887370977),
  'zoom': 13},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJENyBUcmFuc2l0IFN0b3BzIHdpdGhpbiA1MCBtZXRlcnMgb2YgU0hTLCBKdW5lIDIwMjUiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL2FkaG9jZDdfc3RvcHNfMDYyMDI1X3YyLmdlb2pzb24uZ3oiLCAicHJvcGVydGllcyI6IHsic3Ryb2tlZCI6IGZhbHNlLCAiaGlnaGxpZ2h0X3NhdHVyYXRpb25fbXVsdGlwbGllciI6IDAuNX19XSwgImxhdF9sb24iOiBbMzQuMDA5OTY3NDYwMTE3NDY2LCAtMTE4LjM0NzYwODg3MzcwOTc3XSwgInpvb20iOiAxM30='}

In [49]:
gdf.explore()